<a href="https://colab.research.google.com/github/SIDIBEMoussa/LoanPrediction_Zindi_Competition/blob/main/Meilleur_Starter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Africa Credit Challenge

The goals of this `starter notebook` are:

1. How to load data into a `Pandas` dataframe

2. How to `train` a basic machine learning model

3. How to `evaluate` the model

4. How to `save` your __submission__ following the required format.

<font color="red"><b>Note: This is just a starter notebook. It is by no means an ideal solution.</b></font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.chdir("/content/drive/MyDrive/Data_Zindi_Loan/")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

## Load and Inspect the data

In [ ]:
# Loading the train dataset
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
# Display the first few rows of the datasets and their shape
display("Train", train.head(), train.shape, "Test", test.head(), test.shape)

'Train'

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,7,Repeat Loan,120.85,0.014305,121.0,0
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,7,Repeat Loan,7768.50,0.300000,7794.0,0
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,7,Repeat Loan,1380.00,0.200000,1428.0,0
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,7,Repeat Loan,2687.40,0.300000,2770.0,0
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,7,Repeat Loan,1369.20,0.300000,1418.0,0


(68654, 16)

'Test'

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid
0,ID_269404226088267278,269404,Kenya,226088,267278,Type_1,1919.0,1989.0,2022-07-27,2022-08-03,7,Repeat Loan,575.7,0.300000,597.0
1,ID_255356300042267278,255356,Kenya,300042,267278,Type_1,2138.0,2153.0,2022-11-16,2022-11-23,7,Repeat Loan,0.0,0.000000,0.0
2,ID_257026243764267278,257026,Kenya,243764,267278,Type_1,8254.0,8304.0,2022-08-24,2022-08-31,7,Repeat Loan,207.0,0.025079,208.0
3,ID_264617299409267278,264617,Kenya,299409,267278,Type_1,3379.0,3379.0,2022-11-15,2022-11-22,7,Repeat Loan,1013.7,0.300000,1014.0
4,ID_247613296713267278,247613,Kenya,296713,267278,Type_1,120.0,120.0,2022-11-10,2022-11-17,7,Repeat Loan,36.0,0.300000,36.0


(18594, 15)

In [ ]:
# Are there missing values in the train dataset ?
print(f"There are {train.isna().sum().sum()} missing values in the data.")

There are 0 missing values in the data.


In [ ]:
#indicator_data = pd.read_csv('economic_indicators.csv')
#indicator_data.head(2)

In [ ]:
#kenya_indicator = indicator_data.loc[indicator_data.Country == 'Kenya']

In [ ]:
#kenya_indicator.head(1)

In [ ]:
#kenya_indicator.columns = list(map(lambda x: x[2:], indicator_data.columns))

In [ ]:
#kenya_indicator.head(1)

indicator_dict = {}
for indicator_name in kenya_indicator.dicator:
    for year in kenya_indicator.columns[2:]:
        if year>"2020":
            indicator_dict[indicator_name+year] = kenya_indicator.loc[kenya_indicator.dicator == indicator_name, year].values[0]

In [ ]:
def enrich_data(indicator_name,disbursement_date,indicator_dict):

    if float(indicator_name[-4:]) == float(disbursement_date.year):

        return indicator_dict[indicator_name]
    else:
        return np.nan


In [ ]:
def enrich_data(indicator_name, disbursement_date, indicator_dict):
    if indicator_name in indicator_dict:
        year = str(disbursement_date.year)
        if year in indicator_name:
          return indicator_dict[indicator_name]
        else:
          return np.nan
    else:
      return np.nan

In [ ]:
def load_and_preprocess_data(train_path, test_path, indicator_path, inflation_path, cbr_path, interest_path):
    """
    Charge et pré-traite les ensembles de données.

    Args:
        train_path (str): Chemin vers le fichier d'entraînement.
        test_path (str): Chemin vers le fichier de test.
        indicator_path (str): Chemin vers le fichier des indicateurs économiques.
        inflation_path (str): Chemin vers le fichier des taux d'inflation.
        cbr_path(str): Chemin vers le fichier des taux de la banque centrale
        interest_path(str): Chemin vers le fichier des taux d'intérêts bancaires

    Returns:
        tuple: Un tuple contenant les DataFrames train, test, indicator_data,
               inflation_data, cbr, et interest_rate.
    """
    try:
        train = pd.read_csv(train_path)
        test = pd.read_csv(test_path)
        indicator_data = pd.read_csv(indicator_path)
        inflation_data = pd.read_csv(inflation_path)
        cbr = pd.read_csv(cbr_path, header=1)
        interest_rate = pd.read_csv(interest_path, header=1)

        train["disbursement_date"] = pd.to_datetime(train["disbursement_date"])
        train["due_date"] = pd.to_datetime(train["due_date"])

        train["month_int"] = train["disbursement_date"].apply(lambda x: x.month)
        train["day"] = train["disbursement_date"].apply(lambda x: x.day)

        test["disbursement_date"] = pd.to_datetime(test["disbursement_date"])
        test["due_date"] = pd.to_datetime(test["due_date"])

        test["month_int"] = test["disbursement_date"].apply(lambda x: x.month)
        test["day"] = test["disbursement_date"].apply(lambda x: x.day)

        # Conversion des dates
        for df in [train, test]:
            df["disbursement_date"] = pd.to_datetime(df["disbursement_date"])
            df["due_date"] = pd.to_datetime(df["due_date"])

        return train, test, indicator_data, inflation_data, cbr, interest_rate
    except FileNotFoundError:
        print("Erreur : Un ou plusieurs fichiers de données n'ont pas été trouvés.")
        return None, None, None, None, None, None
    except Exception as e:
        print(f"Une erreur est survenue : {e}")
        return None, None, None, None, None, None


def prepare_kenya_indicators(indicator_data):
    """
    Prépare les indicateurs économiques pour le Kenya.

    Args:
        indicator_data (pd.DataFrame): DataFrame des indicateurs économiques.

    Returns:
        tuple: DataFrame des indicateurs pour le Kenya et un dictionnaire
               d'indicateurs.
    """
    kenya_indicator = indicator_data.loc[indicator_data.Country == 'Kenya'].copy()
    kenya_indicator.columns = list(map(lambda x: x[2:], indicator_data.columns))
    indicator_dict = {}
    for indicator_name in kenya_indicator.dicator:
        for year in kenya_indicator.columns[2:]:
            if year > "2020":
                indicator_dict[indicator_name + year] = kenya_indicator.loc[
                    kenya_indicator.dicator == indicator_name, year].values[0]
    return kenya_indicator, indicator_dict

In [ ]:
def enrich_data(indicator_name, disbursement_date, indicator_dict):
    """
    Enrichit les données avec des indicateurs économiques.

    Args:
        indicator_name (str): Nom de l'indicateur.
        disbursement_date (pd.Timestamp): Date de décaissement.
        indicator_dict (dict): Dictionnaire des indicateurs.

    Returns:
        float: La valeur de l'indicateur ou np.nan.
    """
    if indicator_name in indicator_dict:
        year = str(disbursement_date.year)
        if year in indicator_name:
            return indicator_dict[indicator_name]
        else:
            return np.nan
    else:
        return np.nan

In [ ]:
def apply_economic_indicators(df, indicator_dict):
    """
    Applique les indicateurs économiques à un DataFrame.

    Args:
        df (pd.DataFrame): DataFrame à enrichir.
        indicator_dict (dict): Dictionnaire des indicateurs.

    Returns:
        pd.DataFrame: DataFrame enrichi.
    """
    for key in indicator_dict.keys():
        df[key[:-4]] = df.disbursement_date.apply(
            lambda x: enrich_data(key, x, indicator_dict) if x.year < 2024 else np.nan
        )
    return df

In [ ]:
def clean_data(df):
    """
    Nettoie le DataFrame en supprimant certaines colonnes et en ajoutant
    les colonnes 'month' et 'day'.

    Args:
        df (pd.DataFrame): DataFrame à nettoyer.

    Returns:
        pd.DataFrame: DataFrame nettoyé.
    """
    df_cleaned = df
    df_cleaned["month"] = df_cleaned["disbursement_date"].apply(lambda x: x.month)
    df_cleaned["day"] = df_cleaned["disbursement_date"].apply(lambda x: x.day)
    return df_cleaned

In [ ]:
def prepare_inflation_data(inflation_data):
    """
    Prépare les données d'inflation.

    Args:
        inflation_data (pd.DataFrame): DataFrame des données d'inflation.

    Returns:
        pd.DataFrame: DataFrame des données d'inflation préparées.
    """
    Months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November",
              "December"]
    Encodage = {Months[i - 1]: i for i in range(1, 13)}
    inflation_data["month_int"] = inflation_data["Month"].apply(lambda x: Encodage[x])
    inflation_data = inflation_data[inflation_data["Year"] >= 2021]
    return inflation_data

In [ ]:
#prepare_inflation_data(inflation_data)

In [ ]:
import pandas as pd

def join_data_with_inflation(df, inflation_data):
    """
    Joint les données avec les données d'inflation, en préservant le nombre de lignes de df.

    Args:
        df (pd.DataFrame): DataFrame à joindre (le DataFrame de gauche).
        inflation_data (pd.DataFrame): DataFrame des données d'inflation (le DataFrame de droite).

    Returns:
        pd.DataFrame: DataFrame joint avec le même nombre de lignes que df.
    """
    # Créer les colonnes 'Year' et 'month' si elles n'existent pas dans df
    if 'Year' not in df.columns:
        df['Year'] = df['disbursement_date'].dt.year
    if 'month' not in df.columns:
        df['month'] = df['disbursement_date'].dt.month

    # Vérifier si 'month_int' existe dans inflation_data
    #if 'month' not in inflation_data.columns:
    #    raise ValueError("'month_int' column is missing in inflation_data, please check your dataframe")

    # Renommer 'month_int' en 'month'
    inflation_data.rename(columns={'month_int': 'month'}, inplace=True)

    # Vérifier et traiter les doublons dans inflation_data
    if inflation_data.duplicated(subset=['Year', 'month']).any():
        print("Attention : Des doublons ont été détectés dans inflation_data pour les colonnes 'Year' et 'month'.")
        print("Les doublons seront supprimés en gardant la première occurence.")
        # Supprimer les doublons en gardant la première occurrence
        inflation_data.drop_duplicates(subset=['Year', 'month'], keep='first', inplace=True)


    # Effectuer la jointure gauche
    joined_df = pd.merge(df, inflation_data, on=['Year', 'month'], how='left')

    return joined_df


def prepare_cbr_data(cbr):
    """
    Prépare les données des taux de la banque centrale.

    Args:
        cbr (pd.DataFrame): DataFrame des taux de la banque centrale.

    Returns:
        pd.DataFrame: DataFrame des taux de la banque centrale préparés.
    """
    cbr["YEAR"] = pd.to_numeric(cbr["YEAR"], errors='coerce')
    cbr = cbr[cbr["YEAR"] >= 2021]
    cbr = cbr[["YEAR", "Interbank Rate", "Central Bank Rate"]].dropna()
    cbr = cbr.rename(columns={"YEAR": "Year"})
    return cbr


def join_data_with_cbr(df, cbr):
    """
    Joint les données avec les données des taux de la banque centrale.

    Args:
        df (pd.DataFrame): DataFrame à joindre.
        cbr (pd.DataFrame): DataFrame des taux de la banque centrale.

    Returns:
        pd.DataFrame: DataFrame joint.
    """
    return pd.merge(df, cbr, on='Year', how='left')


def prepare_interest_rate_data(interest_rate):
    """
    Prépare les données des taux d'intérêt.

    Args:
        interest_rate (pd.DataFrame): DataFrame des taux d'intérêt.

    Returns:
        pd.DataFrame: DataFrame des taux d'intérêt préparés.
    """
    interest_rate["YEAR"] = pd.to_numeric(interest_rate["YEAR"], errors='coerce')
    interest_rate = interest_rate[interest_rate["YEAR"] >= 2021].dropna()
    interest_rate["interest_rate_spread"] = interest_rate["Lending "] - interest_rate["Deposit"]
    interest_rate = interest_rate[['YEAR', 'MONTH', 'Deposit', 'Savings', 'Lending ', 'Overdraft ',
                                   'interest_rate_spread']].rename(columns={"YEAR": "Year"})
    return interest_rate


def join_data_with_interest_rate(df, interest_rate):
    """
    Joint les données avec les données des taux d'intérêt.

    Args:
        df (pd.DataFrame): DataFrame à joindre.
        interest_rate (pd.DataFrame): DataFrame des taux d'intérêt.

    Returns:
        pd.DataFrame: DataFrame joint.
    """
    return pd.merge(df, interest_rate, on='Year', how='left')


def drop_unnecessary_columns(df):
    """
    Supprime les colonnes inutiles du DataFrame.

    Args:
        df (pd.DataFrame): DataFrame à traiter.

    Returns:
        pd.DataFrame: DataFrame avec les colonnes supprimées.
    """
    na_col = ["Lending interest rate (%)", "Deposit interest rate (%)", "Real interest rate (%)",
              "Official exchange rate (LCU per US$, period average)",
              "Inflation, consumer prices (annual %)", "Month", "Annual Average Inflation",
              "Interest rate spread (lending rate minus deposit rate, %)"]
    return df.drop(columns=na_col)


def fill_missing_values(df):
    """
    Remplit les valeurs manquantes avec des valeurs spécifiques.

    Args:
        df (pd.DataFrame): DataFrame à traiter.

    Returns:
        pd.DataFrame: DataFrame avec les valeurs manquantes remplies.
    """
    df['Unemployment rate'] = df['Unemployment rate'].fillna(6.81)
    df['12-Month Inflation'] = df['12-Month Inflation'].fillna(2.8)
    return df


def convert_central_bank_rate_to_float(df):
    """
    Convertit la colonne "Central Bank Rate" en type float.

    Args:
        df (pd.DataFrame): DataFrame à traiter.

    Returns:
        pd.DataFrame: DataFrame avec la colonne "Central Bank Rate" en float.
    """
    df["Central Bank Rate"] = df["Central Bank Rate"].astype(float)
    return df
def drop_categorical_columns(df,cols):
    return df.drop(columns=cols)

def transform_categorical_columns(df, categorical_columns):
    """
    Transforme les colonnes catégorielles en utilisant get_dummies.

    Args:
        df (pd.DataFrame): DataFrame à traiter.
        categorical_columns (list): Liste des noms de colonnes catégorielles.

    Returns:
        pd.DataFrame: DataFrame avec les colonnes catégorielles transformées.
    """
    return pd.get_dummies(df, columns=categorical_columns, dtype=int)

In [ ]:
# Exemple d'utilisation :
train_path = 'Train.csv'
test_path = 'Test.csv'
indicator_path = 'economic_indicators.csv'
inflation_path = "Inflation Rates.csv"
cbr_path = "410481034_Central Bank Rates.csv"
interest_path = "1041425461_Cormercial banks Weighted Average Rates.csv"

# 1. Charger et pré-traiter les données
train, test, indicator_data, inflation_data, cbr, interest_rate = load_and_preprocess_data(train_path, test_path,
                                                                                              indicator_path,
                                                                                              inflation_path, cbr_path,
                                                                                              interest_path)
samp_sub = pd.read_csv("SampleSubmission.csv")

In [ ]:
#2. Préparer les indicateurs pour le Kenya
kenya_indicator, indicator_dict = prepare_kenya_indicators(indicator_data)

#3. Enrichir les données de train et test avec les indicateurs
train = apply_economic_indicators(train, indicator_dict)
test = apply_economic_indicators(test, indicator_dict)

In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 27), (18594, 26), (18594, 2))

In [ ]:
#4. Nettoyer les données de train et test
train = clean_data(train)
test = clean_data(test)

In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 28), (18594, 27), (18594, 2))

In [ ]:
#5. Préparer les données d'inflation
inflation_data = prepare_inflation_data(inflation_data)

#6. Joindre les données d'inflation aux données de train et test
train = join_data_with_inflation(train, inflation_data)
test = join_data_with_inflation(test, inflation_data)

Attention : Des doublons ont été détectés dans inflation_data pour les colonnes 'Year' et 'month'.
Les doublons seront supprimés en gardant la première occurence.


In [ ]:
inflation_data.head()

,Year,Month,Annual Average Inflation,12-Month Inflation,month
0,2024,October,5.14,2.72,10
1,2024,September,5.50,3.56,9
2,2024,August,5.77,4.36,8
3,2024,July,5.97,4.31,7
4,2024,June,4.64,6.22,6


In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 32), (18594, 31), (18594, 2))

In [ ]:
cbr.head()

,YEAR,MONTH,Repo,Reverse Repo,Interbank Rate,91-Day Tbill,182-days Tbill,364-days Tbill,Cash Reserve Requirement,Central Bank Rate
0,1991,Jul,-,-,NaN,17.14,-,-,-,-
1,NaN,Aug,-,-,NaN,16.70,-,-,-,-
2,NaN,Sep,-,-,NaN,17.18,-,-,-,-
3,NaN,Oct,-,-,NaN,17.78,-,-,NaN,NaN
4,NaN,Nov,-,-,NaN,16.95,-,-,NaN,NaN


In [ ]:
#7. Préparer les données de la banque centrale
cbr = prepare_cbr_data(cbr)

#8. Joindre les données de la banque centrale aux données de train et test
train = join_data_with_cbr(train, cbr)
test = join_data_with_cbr(test, cbr)

In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 34), (18594, 33), (18594, 2))

In [ ]:
#9. Préparer les données des taux d'intérêt
interest_rate = prepare_interest_rate_data(interest_rate)

#10. Joindre les données des taux d'intérêt aux données de train et test
train = join_data_with_interest_rate(train, interest_rate)
test = join_data_with_interest_rate(test, interest_rate)

#11. Supprimer les colonnes inutiles
train = drop_unnecessary_columns(train)
test = drop_unnecessary_columns(test)

#12. Remplir les valeurs manquantes
train = fill_missing_values(train)
test = fill_missing_values(test)

#13. Convertir "Central Bank Rate" en float
train = convert_central_bank_rate_to_float(train)
test = convert_central_bank_rate_to_float(test)

In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 32), (18594, 31), (18594, 2))

In [ ]:
#14. Supprimer les colonnes inutiles pour la suite
cols = ["due_date", "ID", "customer_id", "country_id", "tbl_loan_id", "lender_id"]
train = drop_categorical_columns(train, cols)
#15. transformer les colonnes categorielles en numeriques pour la suite
cate_col = train.select_dtypes('object').columns[:]
train = transform_categorical_columns(train,cate_col)
test = transform_categorical_columns(test,cate_col)

In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 49), (18594, 54), (18594, 2))

In [ ]:
to_drop_col = test.drop(columns='ID').columns.difference(train.columns)
test = test.drop(columns=to_drop_col)

In [ ]:
col_diff = train.drop(columns="target").columns.difference(test.columns)

train = train.drop(columns=col_diff)

In [ ]:
train.shape,test.shape,samp_sub.shape

((68654, 47), (18594, 47), (18594, 2))

In [ ]:
import pandas as pd
import numpy as np

def create_interaction_features(df):
    """
    Creates interaction features by multiplying pairs of existing numerical features.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with new interaction features added.
    """
    df_copy = df.copy()
    numerical_cols = df_copy.select_dtypes(include=np.number).columns.tolist()

    # Avoid combining too many interactions in order to avoid huge numbers of columns
    interactions_to_create = [
        ('loan_amount', 'interest_rate_spread'),
        ('month', 'lending'),
        ('month', 'deposit'),
        ('loan_amount', 'Central Bank Rate'),
    ]

    for col1, col2 in interactions_to_create:
        if col1 in numerical_cols and col2 in numerical_cols:
            df_copy[f'{col1}_x_{col2}'] = df_copy[col1] * df_copy[col2]

    return df_copy

def create_polynomial_features(df, degree=2):
    """
    Creates polynomial features for specified numerical columns.

    Args:
        df (pd.DataFrame): The input DataFrame.
        degree (int): The degree of the polynomial.

    Returns:
        pd.DataFrame: The DataFrame with new polynomial features added.
    """
    df_copy = df.copy()
    numerical_cols = df_copy.select_dtypes(include=np.number).columns.tolist()

    polynomial_cols_to_create = [
        'loan_amount',
        'interest_rate_spread',
    ]

    for col in polynomial_cols_to_create:
        if col in numerical_cols:
            df_copy[f'{col}_poly_{degree}'] = df_copy[col] ** degree

    return df_copy


def create_loan_term_ratio(df):
    """
    Creates a feature representing the loan amount per day.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with the new loan term ratio feature.
    """
    df_copy = df.copy()
    if "disbursement_date" in df_copy.columns and "due_date" in df_copy.columns and 'loan_amount' in df_copy.columns:

        df_copy['loan_amount_per_day'] = df_copy['loan_amount'] / df_copy['duration']
    return df_copy

def create_loan_to_lending_ratio(df):
    """
    Creates a ratio feature of loan amount to lending rate.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with the new ratio feature.
    """
    df_copy = df.copy()
    if "loan_amount" in df_copy.columns and "Lending " in df_copy.columns:
      df_copy["loan_to_lending_ratio"] = df_copy["loan_amount"]/df_copy["Lending "]
    return df_copy

def create_central_bank_rate_change(df):
    """
    Creates a feature to calculate the change in Central Bank Rate compared to the previous year.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with the new Central Bank Rate change feature.
    """
    df_copy = df.copy()

    if "Central Bank Rate" in df_copy.columns:
      df_copy['Year'] = pd.to_datetime(df_copy['disbursement_date']).dt.year
      df_copy['Central Bank Rate Change'] = df_copy.groupby('Year')['Central Bank Rate'].diff()

    return df_copy

def apply_all_features(df):
    """
    Applies all feature engineering functions to the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with all new features added.
    """
    df_copy = df.copy()
    df_copy = create_interaction_features(df_copy)
    df_copy = create_polynomial_features(df_copy)
    df_copy = create_loan_term_ratio(df_copy)
    df_copy = create_loan_to_lending_ratio(df_copy)
    df_copy = create_central_bank_rate_change(df_copy)
    return df_copy

In [ ]:
# Example of how to use the function (assuming you have your `train` and `test` DataFrames):
train = apply_all_features(train)
test = apply_all_features(test)

In [ ]:
train = train.drop(columns=["disbursement_date",'Average precipitation in depth (mm per year)',
       'Fossil fuel energy consumption (% of total)'])
test = test.drop(columns=["disbursement_date",'Average precipitation in depth (mm per year)',
       'Fossil fuel energy consumption (% of total)'])

In [ ]:
train.head()

,Total_Amount,Total_Amount_to_Repay,duration,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target,month_int,day,Unemployment rate,...,loan_type_Type_5,loan_type_Type_6,loan_type_Type_7,loan_type_Type_9,New_versus_Repeat_New Loan,New_versus_Repeat_Repeat Loan,MONTH_JAN,MONTH_JANUARY,interest_rate_spread_poly_2,Central Bank Rate Change
0,8448.0,8448.0,7,120.85,0.014305,121.0,0,8,30,6.81,...,0,0,0,0,0,1,0,1,31.2481,NaN
1,25895.0,25979.0,7,7768.50,0.300000,7794.0,0,7,30,6.81,...,0,0,0,0,0,1,0,1,31.2481,0.0
2,6900.0,7142.0,7,1380.00,0.200000,1428.0,0,9,6,6.81,...,0,0,1,0,0,1,0,1,25.2004,NaN
3,8958.0,9233.0,7,2687.40,0.300000,2770.0,0,10,20,6.81,...,0,0,0,0,0,1,0,1,31.2481,0.0
4,4564.0,4728.0,7,1369.20,0.300000,1418.0,0,11,28,6.81,...,0,0,0,0,0,1,0,1,31.2481,0.0


In [ ]:
#fillna
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
train.to_csv("train_final.csv",index=False)
test.to_csv("test_final.csv",index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

Real: (18594, 15), (68654, 16)

In [ ]:
X_ori,y = train.drop(columns=["target"]), train["target"]

In [ ]:
X_ori

,Total_Amount,Total_Amount_to_Repay,duration,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,month_int,day,Unemployment rate,month,...,loan_type_Type_5,loan_type_Type_6,loan_type_Type_7,loan_type_Type_9,New_versus_Repeat_New Loan,New_versus_Repeat_Repeat Loan,MONTH_JAN,MONTH_JANUARY,interest_rate_spread_poly_2,Central Bank Rate Change
0,8448.0,8448.0,7,120.85,0.014305,121.0,8,30,6.81,8,...,0,0,0,0,0,1,0,1,31.2481,0.0
1,25895.0,25979.0,7,7768.50,0.300000,7794.0,7,30,6.81,7,...,0,0,0,0,0,1,0,1,31.2481,0.0
2,6900.0,7142.0,7,1380.00,0.200000,1428.0,9,6,6.81,9,...,0,0,1,0,0,1,0,1,25.2004,0.0
3,8958.0,9233.0,7,2687.40,0.300000,2770.0,10,20,6.81,10,...,0,0,0,0,0,1,0,1,31.2481,0.0
4,4564.0,4728.0,7,1369.20,0.300000,1418.0,11,28,6.81,11,...,0,0,0,0,0,1,0,1,31.2481,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68649,1460.0,1515.0,7,438.00,0.300000,455.0,7,30,6.81,7,...,0,0,0,0,0,1,0,1,31.2481,0.0
68650,5029.0,5116.0,7,1508.70,0.300000,1535.0,7,16,6.81,7,...,0,0,0,0,0,1,0,1,31.2481,0.0
68651,5289.0,5289.0,7,1586.70,0.300000,1587.0,7,15,6.81,7,...,0,0,0,0,0,1,0,1,31.2481,0.0
68652,3334.0,3334.0,7,741.09,0.222283,741.0,11,23,6.81,11,...,0,0,0,0,0,1,0,1,31.2481,0.0


In [ ]:
#pip install hmmlearn

In [ ]:
import pandas as pd
import numpy as np
from hmmlearn import hmm

def applied_hmm(X):
    """
    Applique un HMM à chaque colonne numérique d'un DataFrame et retourne
    un nouveau DataFrame avec les états cachés prédits. Ignore les colonnes
    pour lesquelles l'HMM ne converge pas ou dont la matrice de transition
    n'est pas valide.

    Args:
        X (pd.DataFrame): Le DataFrame d'entrée.

    Returns:
        pd.DataFrame: Un DataFrame avec les états cachés pour chaque colonne.
    """
    numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
    df_hidden_states = pd.DataFrame()

    for col_name in numerical_cols:
        print(f"Traitement de la colonne : {col_name}")
        # 1. Préparation des données
        X_i = X[col_name]  # On prend la colonne courante

        # Gérer les valeurs manquantes en les remplacant par la medianne.
        if X_i.isnull().any():
            median_value = X_i.median()
            X_i.fillna(median_value, inplace=True)

        # Assurez-vous que X_i est un Series Pandas
        if not isinstance(X_i, pd.Series):
            X_i = pd.Series(X_i)

        # Convertir en tableau NumPy et remodeler
        X_values = X_i.values
        X_reshaped = X_values.reshape(-1, 1)

        # 2. Entraînement du HMM
        # Initialiser et entraîner le modèle HMM
        try:
          model = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100, random_state=42)
          # Initialiser la matrice de transition à des valeurs valides
          model.transmat_ = np.array([[0.5, 0.3, 0.2], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]])
        except ValueError as e:
          print(f"Erreur lors de l'initialisation du modèle pour la colonne {col_name}: {e}")
          continue

        # Ignore les avertissements de convergence
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning, module="hmmlearn")

            # Entraîner le modèle
            try:
              model.fit(X_reshaped)
            except ValueError as e:
              print(f"Erreur lors de l'entrainement du modèle pour la colonne {col_name}: {e}")
              continue
            #Vérifier la convergence
            if not model.monitor_.converged:
                print(f"  Le modèle HMM n'a pas convergé pour la colonne : {col_name}. Colonne ignorée.")
                continue  # Passer à la colonne suivante si le modèle ne converge pas

        # 3. Prédiction des états cachés
        try:
          hidden_states = model.predict(X_reshaped)
        except ValueError as e:
          print(f"Erreur lors de la prédiction des etats cachés pour la colonne {col_name}: {e}")
          continue
        # Ajouter les états cachés au DataFrame de sortie
        df_hidden_states[f'hidden_state_{col_name}'] = hidden_states

    return df_hidden_states


data_enriched = applied_hmm(X_ori)

Traitement de la colonne : Total_Amount


Traitement de la colonne : Total_Amount_to_Repay


Traitement de la colonne : duration


Traitement de la colonne : Amount_Funded_By_Lender


Traitement de la colonne : Lender_portion_Funded


...

Traitement de la colonne : New_versus_Repeat_Repeat Loan


Traitement de la colonne : MONTH_JAN


Traitement de la colonne : MONTH_JANUARY


Traitement de la colonne : interest_rate_spread_poly_2


Traitement de la colonne : Central Bank Rate Change


In [ ]:
# Afficher les données enrichies
data_enriched.shape,X_ori.shape

((68654, 44), (68654, 45))

In [ ]:
X_merged = pd.concat([X_ori, data_enriched], axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_merged, y, test_size=0.2, random_state=42)

## Initialisation & Model Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
report =classification_report(y_valid, model.predict(X_valid))

print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     13487
           1       0.78      0.25      0.38       244

    accuracy                           0.99     13731
   macro avg       0.89      0.63      0.69     13731
weighted avg       0.98      0.99      0.98     13731



In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_valid, label=y_valid)

# Définir les paramètres
params = {
    'objective': 'multi:softmax', # Pour classification multi-classes
    'num_class': 3,              # Nombre de classes
    'max_depth': 4,              # Profondeur maximale de l'arbre
    'eta': 0.3,                  # Taux d'apprentissage
    'seed': 42
}



# Entraîner le modèle
model = xgb.train(params, dtrain, num_boost_round=100)


In [ ]:
y_pred = model.predict(dtest)

# Évaluer la précision
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     13487
           1       0.88      0.60      0.71       244

    accuracy                           0.99     13731
   macro avg       0.94      0.80      0.85     13731
weighted avg       0.99      0.99      0.99     13731



# Optimization

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, classification_report
from numpy import mean
import pandas as pd
from collections import Counter
class_dist = Counter(y_train)
scale_pos_weight = class_dist[0] / class_dist[1]

# Assuming X_train and y_train are your training data and labels
# X_valid and y_valid are your validation set

# Initialize the XGBoost classifier
model = xgb.XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    objective='binary:logistic',  # Specify the objective for binary classification
    eval_metric='auc',
    use_label_encoder=False  # Avoid warning
)

# Define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Convert data to DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)

# Evaluate the model using cross-validation
scores = []
models = []  # to save each model
for train_index, test_index in cv.split(X_train, y_train):
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    class_dist = Counter(y_train_cv)
    scale_pos_weight = class_dist[0] / class_dist[1]
    model_cv = xgb.XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        objective='binary:logistic',  # Specify the objective for binary classification
        eval_metric='auc',
        use_label_encoder=False,  # Avoid warning
        random_state=42,
        n_estimators=400,
        max_depth=5,
        learning_rate=0.3
    )
    model_cv.fit(X_train_cv, y_train_cv)
    models.append(model_cv)

    y_pred_proba = model_cv.predict_proba(X_test_cv)[:, 1]
    scores.append(roc_auc_score(y_test_cv, y_pred_proba))

# Summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.98792


Mean ROC AUC: 0.98644

In [ ]:
#ids = test["ID"]
#test = test.drop(columns=["ID"])
test_created=applied_hmm(test)
test_merged = pd.concat([test, test_created], axis=1)

Traitement de la colonne : Total_Amount


Traitement de la colonne : Total_Amount_to_Repay


Traitement de la colonne : duration


Traitement de la colonne : Amount_Funded_By_Lender


Traitement de la colonne : Lender_portion_Funded


...

Traitement de la colonne : New_versus_Repeat_Repeat Loan


Traitement de la colonne : MONTH_JAN


Traitement de la colonne : MONTH_JANUARY


Traitement de la colonne : interest_rate_spread_poly_2


Traitement de la colonne : Central Bank Rate Change


In [ ]:
test_merged.shape,X_merged.shape

((18594, 90), (68654, 89))

In [ ]:
test_merged.head()

,ID,Total_Amount,Total_Amount_to_Repay,duration,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,month_int,day,Unemployment rate,...,hidden_state_loan_type_Type_5,hidden_state_loan_type_Type_6,hidden_state_loan_type_Type_7,hidden_state_loan_type_Type_9,hidden_state_New_versus_Repeat_New Loan,hidden_state_New_versus_Repeat_Repeat Loan,hidden_state_MONTH_JAN,hidden_state_MONTH_JANUARY,hidden_state_interest_rate_spread_poly_2,hidden_state_Central Bank Rate Change
0,ID_269404226088267278,1919.0,1989.0,7,575.7,0.300000,597.0,7,27,6.81,...,2,2,2,2,2,2,2,2,1,1
1,ID_255356300042267278,2138.0,2153.0,7,0.0,0.000000,0.0,11,16,6.81,...,2,2,2,2,2,2,2,2,1,0
2,ID_257026243764267278,8254.0,8304.0,7,207.0,0.025079,208.0,8,24,6.81,...,2,2,2,2,2,2,2,2,1,1
3,ID_264617299409267278,3379.0,3379.0,7,1013.7,0.300000,1014.0,11,15,6.81,...,2,2,2,2,2,2,2,2,1,0
4,ID_247613296713267278,120.0,120.0,7,36.0,0.300000,36.0,11,10,6.81,...,2,2,2,2,2,2,2,2,1,1


In [ ]:

# you will use the list of model to make a prediction and average it for a better prediction
predictions = []
for model in models:
    predictions.append(model.predict_proba(test_merged.drop(columns=['ID']))[:, 1])  # use all models to do the prediction

# averaging the prediction of all model
average_predictions = mean(predictions, axis=0)

# Create a DataFrame for submission
sub = pd.DataFrame({'ID': test_merged["ID"], 'target': average_predictions})

# Binarize the predictions based on a threshold
# If you want you can play with the value of the threshold
threshold = 0.52
# best 0.52
sub['target'] = (sub['target'] >= threshold).astype(int)

In [ ]:
sub.target.value_counts()

,count
target,
0,17865
1,729


In [ ]:
#target
#0	17729
#1	865

In [ ]:
# save your submission
sub.to_csv('./Submissions/submission_xg_opt_with_sub_data_correction.csv', index=False)

In [ ]:
sub.head()

,ID,target
0,ID_269404226088267278,0
1,ID_255356300042267278,0
2,ID_257026243764267278,0
3,ID_264617299409267278,0
4,ID_247613296713267278,0
